In [4]:
from PIL import Image, ImageDraw
import numpy as np
import math

def pixel_to_view_vector(x, y, width, height):
    """Converts pixel coordinates (equirectangular) to a 3D direction vector."""
    # Normalize coordinates
    nx = x / width
    ny = y / height

    # Calculate longitude (phi) and latitude (theta)
    phi = (nx - 0.5) * 2 * math.pi  # Longitude: -pi to +pi
    theta = (0.5 - ny) * math.pi    # Latitude: +pi/2 (top) to -pi/2 (bottom)

    # Convert spherical to Cartesian coordinates (Y-up convention often used in 3D graphics)
    # Or use Z-up (more common in physics/math) - let's use Z-up for consistency
    # x = cos(theta) * cos(phi)
    # y = cos(theta) * sin(phi)
    # z = sin(theta)
    # Let's use a convention where Z is up, X is forward, Y is left for camera view
    x_cart = math.cos(theta) * math.cos(phi) # Corresponds to forward/backward
    y_cart = math.cos(theta) * math.sin(phi) # Corresponds to left/right
    z_cart = math.sin(theta)                 # Corresponds to up/down

    return np.array([x_cart, y_cart, z_cart])

def visualize_fov_regions(width=1024, height=512, filename="top_fov_visualization.png"):
    """
    Generates an equirectangular image visualizing the approximate coverage
    of four 120-degree FoV cameras pointing towards the top hemisphere.

    Args:
        width (int): Width of the equirectangular image.
        height (int): Height of the equirectangular image (should be width / 2).
        filename (str): Name of the file to save the image.
    """
    if width != height * 2:
        print(f"Warning: Equirectangular images usually have a 2:1 aspect ratio. "
              f"Using width={width}, height={height}.")

    # --- Configuration ---
    fov_deg = 120.0
    fov_rad = math.radians(fov_deg)
    half_fov_rad = fov_rad / 2.0
    cos_half_fov = math.cos(half_fov_rad) # Precompute cosine for efficiency

    # Define camera orientations (Yaw, Pitch) in radians
    # Pitch: Angle up from horizon (0). pi/2 is straight up. Let's use 60 deg = pi/3
    pitch = math.pi / 3.0
    # Yaw: Angle around the vertical axis (0 is often forward)
    yaws = [0, math.pi / 2, math.pi, 3 * math.pi / 2] # 0, 90, 180, 270 degrees

    # Define camera direction vectors (using Z-up convention as in pixel_to_view_vector)
    camera_vectors = []
    for yaw in yaws:
        x_cam = math.cos(pitch) * math.cos(yaw)
        y_cam = math.cos(pitch) * math.sin(yaw)
        z_cam = math.sin(pitch)
        cam_vec = np.array([x_cam, y_cam, z_cam])
        camera_vectors.append(cam_vec / np.linalg.norm(cam_vec)) # Normalize

    # Colors for each camera's FoV (RGBA with alpha for transparency)
    colors = [
        (65, 105, 225, 100),   # Royal Blue (semi-transparent)
        (70, 130, 180, 100),   # Steel Blue (semi-transparent)
        (100, 149, 237, 100),  # Cornflower Blue (semi-transparent)
        (0, 0, 139, 100),      # Dark Blue (semi-transparent)
        # Using different shades of blue like the example image
    ]
    if len(colors) < len(camera_vectors):
        raise ValueError("Need at least as many colors as camera vectors.")

    # --- Image Creation ---
    # Start with a black background
    base_img = Image.new('RGB', (width, height), color='black')
    # Create overlay layers for each camera FoV
    overlay_layers = [Image.new('RGBA', (width, height), (0, 0, 0, 0)) for _ in camera_vectors]
    drawers = [ImageDraw.Draw(layer) for layer in overlay_layers]

    # --- Pixel Processing ---
    # Iterate only over the top half of the image (y < height / 2)
    for y in range(height // 2):
        for x in range(width):
            # Get the direction vector for this pixel
            pixel_vec = pixel_to_view_vector(x, y, width, height)

            # Check if the pixel falls within the FoV of each camera
            for i, cam_vec in enumerate(camera_vectors):
                # Calculate dot product (cosine of angle between vectors)
                dot_product = np.dot(pixel_vec, cam_vec)

                # Check if angle is within half FoV (dot_product >= cos(half_fov))
                # Add small epsilon for floating point comparisons
                if dot_product >= cos_half_fov - 1e-9:
                    # Draw the pixel on the corresponding overlay layer
                    drawers[i].point((x, y), fill=colors[i])

    # --- Compositing ---
    # Alpha composite the layers onto the base image
    final_img = base_img.convert('RGBA') # Convert base to RGBA for compositing
    for layer in overlay_layers:
        final_img = Image.alpha_composite(final_img, layer)

    # --- Saving ---
    # Convert back to RGB if you don't need transparency in the final file
    final_img = final_img.convert('RGB')
    final_img.save(filename)
    print(f"Saved FoV visualization to {filename}")

# --- Run the function ---
image_width = 1024
image_height = image_width // 2

visualize_fov_regions(width=image_width, height=image_height)

# Example with different dimensions
# visualize_fov_regions(width=2048, height=1024, filename="top_fov_visualization_large.png")


Saved FoV visualization to top_fov_visualization.png
